# Hyper-Parameter Tuning for BaggedKNN

In [ ]:
!pip install scikit-optimize
!pip install scikit-learn
!pip install pandas
!pip install catboost

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
from skopt import BayesSearchCV


In [ ]:
x = "your/eeg_features/as/array"
groups = "your/groups/as/array"
y_original = "your/subject_ages/as/array"

y = [int(age*10) for age in y_original]
le = LabelEncoder()
le.fit(y)
y = le.transform(y)

In [ ]:
y_skf = [int(age) for age in y_original]
skf_idx = []
skf = StratifiedGroupKFold(n_splits=3)
for fold, (train_index, test_index) in enumerate(skf.split(x, y_skf, groups)):
    skf_idx.append((train_index, test_index))

In [ ]:
model = RandomForestRegressor(n_jobs=-2)

In [ ]:
parameter_space = {
    'max_depth': [10, 100],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [2, 9],
    'min_weight_fraction_leaf': [0, 0.5],
    'min_impurity_decrease': [0, 0.9],
    'n_estimators': [30, 300]
}

fit_param = {
    'early_stopping_rounds': 200,
}

clf = BayesSearchCV(estimator=model,
                    search_spaces=parameter_space,
                    fit_params=fit_param,
                    cv=skf_idx,
                    scoring='neg_mean_absolute_error',
                    verbose=4)

clf.fit(x, y=y)

In [ ]:
print(clf.cv_results_)
print(clf.best_score_)
print(clf.best_params_)
results = pd.DataFrame(clf.cv_results_)